In [ ]:
import numpy as np
import math
import pygame
import sys
import random
import copy
import pylab
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
FPS = 60
BLACK = (0, 0, 0)
WITHE = (255, 255, 255)
YELLOW = (255, 255, 0)
VISION = 100
HIGHT = 1000
WIGHT = 1000
LAYER_RUN = [4, 5, 2]
LAYER_SKILLS = [4, 5, 3]

In [ ]:
class AI(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.genom = [np.random.rand(LAYER_RUN[i], LAYER_RUN[i + 1]) for i in range(len(LAYER_RUN) - 1)]
        self.genomAction = [np.random.rand(LAYER_SKILLS[i], LAYER_SKILLS[i + 1]) for i in range(len(LAYER_SKILLS) - 1)]
        self.x = random.randint(0, HIGHT)
        self.y = random.randint(0, WIGHT)
        self.xV = 0
        self.yV = 0
        self.food = random.randint(0, 10)
        self.damage = random.randint(0, 10)
        self.defend = random.randint(0, 10)
        self.energy = 100
        self.image = pygame.transform.scale(pygame.image.load('resource/individ.png').convert_alpha(), (20, 20))
        self.rect = self.image.get_rect(center=(self.x, self.y))
        self.time = 0
        self.childs = 0
    
    def f(self, x):
        return (math.e ** x - math.e ** (-x)) / (math.e ** x + math.e ** (-x))
    
    def calculate(self, inputValue):
        sig = np.vectorize(self.f)
        hiddenValue = np.matrix(sig(inputValue * self.genom[0]))
        outputValue = sig(hiddenValue * self.genom[1])
        #print(outputValue)
        return np.array(outputValue)
    
    def do(self, inputValue):
        outputValue = self.calculate(inputValue)
        outputValue[0].argmax()
        self.xV += outputValue[0][0] if self.xV <= 10 else 0
        self.yV += outputValue[0][1] if self.yV <= 10 else 0
        self.x += self.xV
        self.y += self.yV
        self.rect.x = self.x
        self.rect.y = self.y
        self.energy -= 2
        self.time += 1
        
    def eat(self):
        self.energy += 50
    
    def takeDamage(self, value):
        self.energy -= (value - self.defend) * 0 if value > self.defend else 0
        
    def giveDamage(self, individs):
        [individ.takeDamage(self.damage) for individ in individs]
    
    def copy(self):
        newIndivid = AI()
        newIndivid.x = self.x + 10
        newIndivid.y = self.y + 10
        newIndivid.genom = [list(self.genom[0]), list(self.genom[1])]
        rand = random.randint(0, 100)
        if rand < 5:
            newIndivid.genom[0][random.randint(0, LAYER_RUN[0] - 1)][random.randint(0, LAYER_RUN[1] - 1)] = random.random()
        elif rand > 5 and rand < 10:
            newIndivid.genom[1][random.randint(0, LAYER_RUN[1] - 1)][random.randint(0, LAYER_RUN[2] - 1)] = random.random()
        newIndivid.genom = [np.array(newIndivid.genom[0]), np.array(newIndivid.genom[1])]
        newIndivid.energy = self.energy // 2
        self.energy //= 2
        self.childs += 1
        return newIndivid

In [ ]:
class Food(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.x = np.random.randint(0, HIGHT)
        self.y = np.random.randint(0, WIGHT)
        self.image = pygame.transform.scale(pygame.image.load('resource/food.jpg').convert_alpha(), (10, 10))
        self.rect = self.image.get_rect(center=(self.x, self.y))

In [ ]:
class Manager:
    def __init__(self):
        self.individs = [AI() for i in range(40)]
        self.foods = [Food() for i in range(200)]
        
    def dist(self, a, b):
        return ((a.x - b.x) ** 2 + (a.y - b.y) ** 2) ** 0.5
    
    def beside(self, ai):
        return [individ for individ in self.individs if self.dist(ai, individ) <= VISION and ai != individ]
    
    def data(self, objects):
        vis = [[np.array([j.x, j.y]) for j in objects if self.dist(i, j) <= VISION and i != j] for i in self.individs]
        v = [sum(i) // len(i) if len(i) != 0 else np.array([0, 0]) for i in vis]
        return v
    
    def eat(self, per):
        [[self.foods.remove(i), j[0].eat()] for i, j in per.items()]
        
    def step(self):
        inputEats = self.data(self.foods)
        inputIndivids = self.data(self.individs)
        [ai.do(np.matrix([(eat[0] - ai.x) / VISION, (eat[1] - ai.y) / VISION, (individ[0] - ai.x) / VISION, (individ[1] - ai.y) / VISION])) for ai, eat, individ in zip(self.individs, inputEats, inputIndivids)]
        [ai.giveDamage(self.beside(ai)) for ai in self.individs]
        self.eat(pygame.sprite.groupcollide(self.foods, self.individs, False, False, pygame.sprite.collide_mask))
        [self.individs.remove(i) for i in self.individs if i.energy <= 0]
        [self.individs.append(i.copy()) for i in self.individs if i.energy >= 80]
        self.foods.extend([Food() for i in range(1)])
        return self.individs, self.foods, max(self.individs, key = lambda x : x.childs) if len(self.individs) != 0 else AI()

In [ ]:
def engEvent(): 
    for i in pygame.event.get(): 
        if i.type == pygame.QUIT: 
            pygame.quit() 
            sys.exit()

In [ ]:
%matplotlib
pygame.init()
screen = pygame.display.set_mode((HIGHT, WIGHT), pygame.HWSURFACE|pygame.DOUBLEBUF) 
pygame.display.set_caption("My Game") 
clock = pygame.time.Clock()

manager = Manager()

best = AI()

x = []
y = []

t = 0
while 1: 
    engEvent()
    clock.tick(300) 
    individs, foods, better = manager.step()
    
    best = better if best.childs < better.childs else best
    

     
    [screen.blit(i.image, i.rect) for i in individs]
    
    [screen.blit(i.image, i.rect) for i in foods]
    
    if len(individs) <= 0:
        manager = Manager()
    
    x.append(t * 10)
    y.append(best.childs)
    
    plt.plot(x, y)
    
    plt.draw()
    
    t += 0.1 
    pygame.display.flip() 
    screen.fill(WITHE)
plt.show()